# Credit Scoring Feed

This notebook can be scheduled to provide a continuous flow of data to your German Credit Risk model. Paste your Watson Machine Learning credentials below, and verify that the value of the **DEPLOYMENT_NAME** variable matches the name of your deployed German Credit Risk model. You can also set the maximum and minimum number of records that will be scored per run.

In [1]:
!pip install --upgrade watson-machine-learning-client --no-cache | tail -n 1

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20190612231939-0000
KERNEL_ID = b69d525a-6c44-4f41-8f1a-01b8e0842c06
tensorflow 1.13.1 requires tensorboard<1.14.0,>=1.13.0, which is not installed.
spyder 3.3.3 requires pyqt5<=5.12; python_version >= "3", which is not installed.
ibm-cos-sdk-core 2.4.4 has requirement urllib3<1.25,>=1.20, but you'll have urllib3 1.25.3 which is incompatible.
botocore 1.12.82 has requirement urllib3<1.25,>=1.20, but you'll have urllib3 1.25.3 which is incompatible.


In [2]:
WML_CREDENTIALS = {
  "apikey": "DLWWjSw5Vttta3k4kccmlPqAo2otR8-3FuQRLDDUiRWE",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:pm-20:us-south:a/6bf3c9070c42bf0f4ea682367d8f4f81:e2a27a67-8c2c-43d3-92b6-efcded30a28b::",
  "iam_apikey_name": "auto-generated-apikey-a93f2ba3-750d-4b7b-97b7-a380d315cd1c",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/6bf3c9070c42bf0f4ea682367d8f4f81::serviceid:ServiceId-1e9dd134-cbc8-4c23-b606-d3330ab35c66",
  "instance_id": "e2a27a67-8c2c-43d3-92b6-efcded30a28b",
  "password": "d53b23fe-3f18-4dfe-bc36-da42654bf472",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "a93f2ba3-750d-4b7b-97b7-a380d315cd1c"
}

In [3]:
DEPLOYMENT_NAME = "Risk Deployment - Final"
MIN_RECORDS = 30
MAX_RECORDS = 200

In [4]:
import json, random
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [5]:
wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

In [6]:
wml_deployments = wml_client.deployments.get_details()
scoring_url = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        scoring_url = deployment['entity']['scoring_url']
        break
    
print("Scoring URL: {}".format(scoring_url))

Scoring URL: https://us-south.ml.cloud.ibm.com/v3/wml_instances/e2a27a67-8c2c-43d3-92b6-efcded30a28b/deployments/e0912891-3d18-43ae-94a3-394e11442100/online


In [7]:
try:
    with open('german_credit_feed.json', 'r') as scoring_file:
        scoring_data = json.load(scoring_file)
    print('file found')
    
except:
    !wget https://raw.githubusercontent.com/emartensibm/german-credit/master/german_credit_feed.json
    with open('german_credit_feed.json', 'r') as scoring_file:
        scoring_data = json.load(scoring_file)
    print('file downloaded')

--2019-06-12 23:25:20--  https://raw.githubusercontent.com/emartensibm/german-credit/master/german_credit_feed.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3076547 (2.9M) [text/plain]
Saving to: 'german_credit_feed.json'

german_credit_feed. 100%[===================>]   2.93M  --.-KB/s    in 0.06s   

2019-06-12 23:25:20 (50.8 MB/s) - 'german_credit_feed.json' saved [3076547/3076547]

file downloaded


In [8]:
fields = scoring_data['fields']
values = []
for _ in range(0, random.randint(MIN_RECORDS, MAX_RECORDS)):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"fields": fields, "values": values}

scoring_response = wml_client.deployments.score(scoring_url, payload_scoring)
print(scoring_response)

{'fields': ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'CheckingStatus_IX', 'CreditHistory_IX', 'EmploymentDuration_IX', 'ExistingSavings_IX', 'ForeignWorker_IX', 'Housing_IX', 'InstallmentPlans_IX', 'Job_IX', 'LoanPurpose_IX', 'OthersOnLoan_IX', 'OwnsProperty_IX', 'Sex_IX', 'Telephone_IX', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'], 'values': [['0_to_200', 16, 'credits_paid_to_date', 'car_new', 4946, 'less_100', '4_to_7', 4, 'female', 'none', 2, 'unknown', 41, 'none', 'own', 1, 'skilled', 1, 'none', 'yes', 2.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 1.0, 0.0, [2.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 4946.0, 4.0, 2.0, 16.0, 41.0, 1.0, 1.